In [ ]:
##Step 0 : Load Libraries 

In [63]:
# LIBRARIES ----

import pandas as pd
import numpy as np
import pytimetk as tk

from pymc_marketing.mmm.delayed_saturated_mmm import DelayedSaturatedMMM

import arviz as az
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
## Step 1: Load and Explore data

In [15]:

df = pd.read_csv("https://raw.githubusercontent.com/analytic-nick/marketing_analytics/main/mmm/data/de_simulated_data.csv", parse_dates = ["DATE"])


In [ ]:
## Step 2 : Feature Engineering

In [19]:

# Time Series Features
df_features = df \
    .assign(
        year = lambda x: x["DATE"].dt.year,
        month = lambda x: x["DATE"].dt.month,
        dayofyear = lambda x: x["DATE"].dt.dayofyear,
    ) \
    .assign(
        trend = lambda x: df.index,
    ) 

df_features = df_features[['DATE', 'revenue', 'tv_S', 'ooh_S', 'print_S', 'facebook_S', 'search_S', 'trend', 'year', 'month', 'dayofyear']]


In [ ]:
## Step 3: Model Specifications
#  - Reference: https://www.pymc-marketing.io/en/stable/notebooks/mmm/mmm_example.html#id1
#  - DelayedSaturatedMMM handles scaling transformations internally
#  - Uses MaxAbsScaler transformer from scikit-learn
#  - Specify the priors in the scaled space (i.e. between 0 and 1)
#  - One way to do it is to use spend / max(spend) as the prior

In [21]:
# * Create Priors from Business Knowledge

total_spend_per_channel = df_features[['tv_S', 'ooh_S', 'print_S', 'facebook_S', 'search_S']].sum(axis=0)

spend_proportion = total_spend_per_channel / total_spend_per_channel.sum()

HALFNORMAL_SCALE = 1 / np.sqrt(1 - 2 / np.pi)

n_channels = 5

prior_sigma = HALFNORMAL_SCALE * n_channels * spend_proportion

prior_sigma.tolist()

[2.8061740654561564,
 2.042569187944328,
 0.7048914591334415,
 1.6225315542033876,
 1.1183174331142183]

## Set up Model Configurations

In [24]:
# * Create a Model Specification

X = df_features.drop("revenue",axis=1)
y = df_features["revenue"]

# Default Model Configuration

dummy_model = DelayedSaturatedMMM(
    date_column = "date",
    channel_columns = ['tv_s', 'ooh_s', 'print_s', 'facebook_s', 'search_s'],
    control_columns = [
        "trend",
        "year",
        "month",
    ],
    adstock_max_lag=8,
)

dummy_model.default_model_config

# Customizing the Model Configuration

my_model_config = {
    'intercept': {
        'dist': 'Normal', 
        'kwargs': {
            'mu': 0, 
            'sigma': 2
        }
    },
    'beta_channel': {
        'dist': 'HalfNormal', 
        'kwargs': {
            'sigma': 2
        }
    },
    # 'beta_channel': {
    #     'dist': 'LogNormal',
    #     "kwargs": {
    #         "mu": np.array([5,1]), 
    #         "sigma": prior_sigma.to_numpy()
    #     }
    # },
    "likelihood": {
        "dist": "Normal",
        "kwargs":{
            "sigma": {
                'dist': 'HalfNormal', 
                'kwargs': {
                    'sigma': 2
                }
            }
        }
    },
    'alpha': {
        'dist': 'Beta', 
        'kwargs': {'alpha': 1, 'beta': 3}
    },
    'lam': {
        'dist': 'Gamma', 
        'kwargs': {'alpha': 3, 'beta': 1}
    },
    'gamma_control': {
        'dist': 'Normal', 
        'kwargs': {'mu': 0, 'sigma': 2}
    },
    'gamma_fourier': {
        'dist': 'Laplace', 
        'kwargs': {'mu': 0, 'b': 1}
    },
}

my_sampler_config= {
    "progressbar": True,
    "cores": 1,
}

C:\Users\nfole\AppData\Local\Temp\ipykernel_20096\1747854405.py:8: DeprecationWarning: The DelayedSaturatedMMM class is deprecated. Please use the MMM class instead.
  dummy_model = DelayedSaturatedMMM(


## Step 4: Model Fitting

In [26]:
# * DelayedSaturatedMMM Model

mmm = DelayedSaturatedMMM(
    model_config = my_model_config,
    sampler_config = my_sampler_config,
    date_column = "DATE",
    channel_columns = ['tv_S', 'ooh_S', 'print_S', 'facebook_S', 'search_S'],
    control_columns = [
        "trend",
        "year",
        "month",
    ],
    adstock_max_lag=8,
    yearly_seasonality=2,
)

mmm.model_config

mmm.default_model_config



# Fit the Model
mmm.fit(X,y, target_accept = 0.95, random_seed = 888)

# Production Model Storage and Retrieval
loaded_mmm =mmm

C:\Users\nfole\AppData\Local\Temp\ipykernel_20096\2548858965.py:7: DeprecationWarning: The DelayedSaturatedMMM class is deprecated. Please use the MMM class instead.
  mmm = DelayedSaturatedMMM(
C:\Users\nfole\Anaconda3\envs\pymc_env\Lib\site-packages\pymc\model\core.py:518: FutureWarning: All coords are now mutable by default. coords_mutable will be removed in a future release.
  warnings.warn(
C:\Users\nfole\Anaconda3\envs\pymc_env\Lib\site-packages\pymc\model\core.py:988: FutureWarning: Coords are now always mutable. Specifying `mutable` will raise an error in a future release
  warnings.warn(
C:\Users\nfole\Anaconda3\envs\pymc_env\Lib\site-packages\pymc\data.py:423: FutureWarning: Data is now always mutable. Specifying the `mutable` kwarg will raise an error in a future release
  warnings.warn(
C:\Users\nfole\Anaconda3\envs\pymc_env\Lib\site-packages\pymc\data.py:423: FutureWarning: Data is now always mutable. Specifying the `mutable` kwarg will raise an error in a future release
 

Output()

Output()

Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 3735 seconds.
Chain 0 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain 1 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data
	> fit_data

## Step 5: Analyze Model Results

In [30]:
loaded_mmm=mmm

In [34]:
# Plot Components Contributions

#fig = loaded_mmm.plot_components_contributions()

# Plot Graphical MMM Model
#fig = loaded_mmm.graphviz()
#fig

# Plot Direct Contribution Curves
#fig = loaded_mmm.plot_direct_contribution_curves()

# Plot Channel Contributions
# Takes a bit to run

#fig = loaded_mmm.plot_channel_contributions_grid(start=0, stop=1.5, num=12, absolute_xrange=True);


## Step 6: Translate to Business Results

In [36]:

# Estimate Return on Ad Spend (ROAS) by Channel

get_mean_contributions_over_time_df = loaded_mmm.compute_mean_contributions_over_time(original_scale=True)

channel_contribution_original_scale = loaded_mmm.compute_channel_contribution_original_scale()

roas_samples = (
    channel_contribution_original_scale.stack(sample=("chain", "draw")).sum("date")
    / X[['tv_S', 'ooh_S', 'print_S', 'facebook_S', 'search_S']].sum().to_numpy()[..., None]
)


In [37]:
roas_samples

<xarray.DataArray (channel: 5, sample: 2000)> Size: 80kB
array([[0.54630722, 0.61150943, 0.589891  , ..., 0.82867029, 0.57355039,
        0.75603342],
       [0.62912845, 0.51618589, 0.54769601, ..., 0.21242511, 0.45969538,
        0.68504043],
       [0.23260216, 1.92243415, 0.26261533, ..., 1.44954566, 0.3701113 ,
        0.57566733],
       [1.53641802, 0.4008956 , 0.17537405, ..., 1.23883499, 1.29032654,
        0.15773184],
       [3.14229869, 0.63204227, 0.63338553, ..., 2.13199704, 3.25942198,
        5.40548075]])
Coordinates:
  * channel  (channel) <U10 200B 'tv_S' 'ooh_S' ... 'facebook_S' 'search_S'
  * sample   (sample) object 16kB MultiIndex
  * chain    (sample) int32 8kB 0 0 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1 1 1 1
  * draw     (sample) int32 8kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999